In [1]:
!pip install deepxde tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 10.6 MB/s eta 0:00:00


In [2]:
import deepxde as dde
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split


No backend selected.
Finding available backend...


Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.
Instructions for updating:
non-resource variables are not supported in the long term


Found tensorflow.compat.v1
Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)


Enable just-in-time compilation with XLA.



In [3]:
# Set random seed for reproducibility
dde.config.set_random_seed(42)

# Load the Ansys data
file_path = "/content/dataset_total_fin (1).csv"
data_df = pd.read_csv(file_path)

# Extract and convert coordinates from meters to millimeters
# Strip leading/trailing whitespaces from column names
data_df.columns = data_df.columns.str.strip()

# Access 'time', 'x-coordinate', 'y-coordinate', 'temperature' columns
# (after stripping)
time_steps = data_df['Time'].values  # Seconds
x_coords = data_df['x-coordinate'].values * 1000  # Convert m to mm
y_coords = data_df['y-coordinate'].values * 1000  # Convert m to mm
temperatures = data_df['temperature'].values  # Kelvin

# OR, Access the column by its index if you know it (e.g., 0th column):
# time_steps = data_df.iloc[:, 0].values # access the 0th column

# Prepare data for model
X = np.column_stack((x_coords, y_coords, time_steps))
y = temperatures.reshape(-1, 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Repeat training data to emphasize it (mimics weighting)
X_train = np.repeat(X_train, 5, axis=0)
y_train = np.repeat(y_train, 5, axis=0)

In [6]:
# Define thermal diffusivity of copper (mm²/s)
alpha = 117.0

# Define the computational geometry and time domain
geom = dde.geometry.Rectangle([-10, -10], [10, 10])
timedomain = dde.geometry.TimeDomain(0, 8)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# Define the PDE (heat equation)
# Define the PDE (heat equation)
def pde(x, y):
    dy_t = dde.grad.jacobian(y, x, i=0, j=2)  # ∂T/∂t
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)  # ∂²T/∂x²
    dy_yy = dde.grad.hessian(y, x, i=1, j=1)  # ∂²T/∂y²
    return dy_t - alpha * (dy_xx + dy_yy)

In [7]:
# Define boundary and initial conditions
def initial_condition(x, on_initial):
    return on_initial and np.isclose(x[2], 0)

def top_boundary(x, on_boundary):
    return on_boundary and np.isclose(x[1], 10)  # y = 10 mm

def left_boundary(x, on_boundary):
    return on_boundary and np.isclose(x[0], -10)  # x = -10 mm

def right_boundary(x, on_boundary):
    return on_boundary and np.isclose(x[0], 10)  # x = 10 mm

def bottom_boundary(x, on_boundary):
    return on_boundary and np.isclose(x[1], -10)  # y = -10 mm

# Define boundary and initial conditions
ic = dde.IC(geomtime, lambda x: 300, initial_condition)  # T = 300 K at t = 0
bc_top = dde.DirichletBC(geomtime, lambda x: 400, top_boundary)  # T = 400 K at y = 10 mm
bc_left = dde.NeumannBC(geomtime, lambda x: 50, left_boundary)  # Neumann: ∂T/∂x = 50 K/mm
bc_right = dde.NeumannBC(geomtime, lambda x: 0, right_boundary)  # Insulated
bc_bottom = dde.NeumannBC(geomtime, lambda x: 0, bottom_boundary) # Insulated

In [8]:
# Create observation data from training set
observe_x = X_train  # In mm: [x, y, t]

# Define corners with time dimension
corners = [[-10, -10, 0], [-10, 10, 0], [10, -10, 0], [10, 10, 0]]  # Add time=0

# Create the data object with all constraints
data = dde.data.TimePDE(
    geomtime,
    pde,
    [ic, bc_top, bc_left, bc_right, bc_bottom, dde.icbc.PointSetBC(observe_x, y_train)],
    num_domain=25000,    # points
    num_boundary=10000,   #  boundary points
    num_initial=5000,    #  initial points
    anchors=observe_x, # Include data points
    exclusions=corners
)

# Network architecture (wider for complex fields)
layer_size = [3] + [256] * 5 + [128] * 4 + [64] * 3 + [32] * 2 + [1]
activation = "relu"
initializer = "Glorot normal"
net = dde.nn.FNN(layer_size, activation, initializer)

# Define the model
model = dde.Model(data, net)

In [15]:
# Compile the model with lower learning rate
model.compile("adam", lr=0.003)

# Train with more iterations
losshistory, train_state = model.train(iterations=50000)

# Switch to L-BFGS for refinement
model.compile("L-BFGS")
losshistory, train_state = model.train(iterations=3000)

Compiling model...
'compile' took 0.860919 s

Training model...

Step      Train loss                                                                Test loss                                                                 Test metric
10017     [3.11e+02, 3.58e+02, 4.74e+02, 2.26e+03, 1.24e+01, 1.16e+01, 1.01e+03]    [3.11e+02, 3.58e+02, 4.74e+02, 2.26e+03, 1.24e+01, 1.16e+01, 1.01e+03]    []  
11000     [2.91e+02, 2.63e+02, 2.06e+01, 1.54e+03, 8.80e+00, 3.50e+00, 3.99e+02]    [2.91e+02, 2.63e+02, 2.06e+01, 1.54e+03, 8.80e+00, 3.50e+00, 3.99e+02]    []  
12000     [3.25e+02, 1.86e+02, 6.54e+01, 2.27e+03, 8.00e-01, 3.30e+00, 1.61e+02]    [3.25e+02, 1.86e+02, 6.54e+01, 2.27e+03, 8.00e-01, 3.30e+00, 1.61e+02]    []  
13000     [3.12e+02, 1.64e+02, 2.46e+01, 2.27e+03, 2.18e+00, 1.25e+00, 1.44e+02]    [3.12e+02, 1.64e+02, 2.46e+01, 2.27e+03, 2.18e+00, 1.25e+00, 1.44e+02]    []  
14000     [3.32e+02, 9.88e+02, 1.42e+02, 1.89e+03, 3.82e+00, 2.27e+00, 2.66e+02]    [3.32e+02, 9.88e+02, 1.42e+02

In [16]:
# Plot and save loss history
dde.utils.plot_loss_history(losshistory)
plt.title("Training Loss History")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.savefig("loss_history_adjusted.png")
plt.close()

In [18]:
# Generate predictions on a finer grid
all_points = []
all_predictions = []

for t in range(1, 9):
    # Create a grid: 200x200 for ~0.1 mm resolution
    x_grid = np.linspace(-10, 10, 50)
    y_grid = np.linspace(-10, 10, 50)
    X_grid, Y_grid = np.meshgrid(x_grid, y_grid)

    # Create input points
    points = np.zeros((50*50, 3))
    points[:, 0] = X_grid.flatten()
    points[:, 1] = Y_grid.flatten()
    points[:, 2] = t

    # Make predictions
    T_pred = model.predict(points)

    # Store points and predictions
    for i in range(len(points)):
        all_points.append(points[i])
        all_predictions.append(T_pred[i][0])

In [19]:
# Create a DataFrame for the results
results_df = pd.DataFrame({
    'Time_step': [p[2] for p in all_points],
    'X_Coordinate': [p[0] for p in all_points],
    'Y_Coordinate': [p[1] for p in all_points],
    'Predicted_Temperature': all_predictions
})

# Save the results to a CSV file
results_df.to_csv("pinn_predictions_adjusted.csv", index=False)

In [20]:
# Evaluate on training and test sets
train_pred = model.predict(X_train)
train_mae = np.mean(np.abs(train_pred - y_train))
test_pred = model.predict(X_test)
test_mae = np.mean(np.abs(test_pred - y_test))
test_mse = np.mean((test_pred - y_test) ** 2)
print(f"Train MAE: {train_mae:.2f} K")
print(f"Test MSE: {test_mse:.2f} K²")
print(f"Test MAE: {test_mae:.2f} K")

# Compare specific Ansys points
ansys_points = np.array([
    [6, 2, 1],    # Ansys: 382.0570584 K
    [-6, -8, 1],   # Ansys: 400 K
    [0, 0, 1],   # Ansys: 378.4155069 K
    [ 0, -2, 2],
    [8, 0, 2]
])
ansys_temps = np.array([382.0570584, 400.0, 378.4155069, 355.8986289, 375.3037934, 378.414543])
pred_temps = model.predict(ansys_points)
for i, (pt, ansys_t, pred_t) in enumerate(zip(ansys_points, ansys_temps, pred_temps)):
    print(f"Point {i+1}: ({pt[0]}, {pt[1]}, Time={pt[2]}) | Ansys: {ansys_t:.2f} K | PINN: {pred_t[0]:.2f} K | Error: {abs(ansys_t - pred_t[0]):.2f} K")

Train MAE: 7.47 K
Test MSE: 218.78 K²
Test MAE: 9.90 K
Point 1: (6, 2, Time=1) | Ansys: 382.06 K | PINN: 338.97 K | Error: 43.09 K
Point 2: (-6, -8, Time=1) | Ansys: 400.00 K | PINN: 329.58 K | Error: 70.42 K
Point 3: (0, 0, Time=1) | Ansys: 378.42 K | PINN: 344.90 K | Error: 33.52 K
Point 4: (0, -2, Time=2) | Ansys: 355.90 K | PINN: 348.09 K | Error: 7.81 K
Point 5: (8, 0, Time=2) | Ansys: 375.30 K | PINN: 369.74 K | Error: 5.57 K


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata

# Load datasets
try:
    df_ansys = pd.read_csv("dataset_total_fin (1).csv", header=0)
    print("ANSYS raw data (first 5 rows):")
    print(df_ansys.head())
    print("ANSYS raw columns:", df_ansys.columns.tolist())

    df_ansys.columns = ["Time", "x", "y", "temperature"]
    df_ansys.rename(columns={"temperature": "temperature_ansys"}, inplace=True)
except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure 'dataset_total_fin.csv' is in the correct directory.")
    exit()
except Exception as e:
    print(f"Error loading ANSYS data: {e}")
    exit()

try:
    df_pinn = pd.read_csv("/content/pinn_predictions_adjusted (2) (1).csv", header=None, names=["Time", "x", "y", "temperature"])
    df_pinn.rename(columns={"temperature": "temperature_pinn"}, inplace=True)
    df_boundary = pd.read_csv("dataset_boundary (1).csv", header=0, names=["Time", "x", "y", "temperature"])
    df_boundary.rename(columns={"temperature": "temperature_boundary"}, inplace=True)
except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure PINN and boundary CSV files are in the correct directory.")
    exit()

# Convert ANSYS coordinates from meters to millimeters
df_ansys['x'] = df_ansys['x'] * 1000  # Convert meters to millimeters
df_ansys['y'] = df_ansys['y'] * 1000  # Convert meters to millimeters

# Convert all columns to numeric, handling invalid entries
df_ansys = df_ansys.apply(pd.to_numeric, errors='coerce')
df_pinn = df_pinn.apply(pd.to_numeric, errors='coerce')
df_boundary = df_boundary.apply(pd.to_numeric, errors='coerce')

# Debug: Check data before dropping NaNs
print("\nANSYS data before dropna:")
print(df_ansys.head())
print("ANSYS Time values unique:", df_ansys['Time'].unique())

# Drop rows with NaN values
df_ansys = df_ansys.dropna()
df_pinn = df_pinn.dropna()
df_boundary = df_boundary.dropna()

# Debug: Check data after dropping NaNs
print("\nANSYS data after dropna:")
print(df_ansys.head())
print("ANSYS data shape:", df_ansys.shape)

# Filter for time steps 1 to 8
df_ansys = df_ansys[df_ansys["Time"].between(1, 8)]
df_pinn = df_pinn[df_pinn["Time"].between(1, 8)]
df_boundary = df_boundary[df_boundary["Time"].between(1, 8)]

# Debug: Check data after time filtering
print("\nANSYS data after time filtering (Time 1-8):")
print(df_ansys.head())
print("ANSYS data shape after filtering:", df_ansys.shape)
print("ANSYS Time values after filtering:", df_ansys['Time'].unique())

# Check if ANSYS data is empty
if df_ansys.empty:
    print("Error: ANSYS data is empty after filtering. Possible issues:")
    print("- Time column does not contain values between 1 and 8.")
    print("- All data was NaN after numeric conversion.")
    print("- CSV file may have incorrect format or missing data.")
    exit()

# Debug: Check ANSYS and PINN data range
print("\nANSYS data summary:")
print(df_ansys[["Time", "temperature_ansys"]].groupby("Time").agg(["min", "max", "mean"]))
print("\nPINN data summary:")
print(df_pinn[["Time", "temperature_pinn"]].groupby("Time").agg(["min", "max", "mean"]))

# Interpolation function with fixed -10 to 10 mm range
def interpolate_data(df, time, value_col, grid_res=200):
    df_t = df[df["Time"] == time]
    if df_t.empty:
        raise ValueError(f"No data found for time step {time}")
    # Debug: Check number of points
    print(f"Time {time}: {value_col} - Number of points = {len(df_t)}")
    # Check for duplicates
    duplicates = df_t.duplicated(subset=["x", "y"]).sum()
    if duplicates > 0:
        print(f"Warning: {duplicates} duplicate (x, y) points found for time {time}. Taking mean.")
        df_t = df_t.groupby(["x", "y"]).agg({value_col: "mean"}).reset_index()
    # Use fixed coordinate range: -10 to 10 mm
    x_min, x_max = -10, 10
    y_min, y_max = -10, 10
    print(f"Time {time}: {value_col} - x range = ({x_min:.4f}, {x_max:.4f}), y range = ({y_min:.4f}, {y_max:.4f})")
    xi = np.linspace(x_min, x_max, grid_res)
    yi = np.linspace(y_min, y_max, grid_res)
    X, Y = np.meshgrid(xi, yi)
    Z = griddata((df_t["x"], df_t["y"]), df_t[value_col], (X, Y), method='linear')
    # Handle NaNs
    if np.any(np.isnan(Z)):
        print(f"Warning: NaN values in interpolated data for {value_col} at time {time}. Filling with nearest.")
        Z = griddata((df_t["x"], df_t["y"]), df_t[value_col], (X, Y), method='nearest')
    return X, Y, Z

# Generate heatmaps for each time step
for t in range(1, 9):
    try:
        # Interpolate data for ANSYS and PINN
        grid_res = 200  # Same as before
        X_ansys, Y_ansys, Z_ansys = interpolate_data(df_ansys, t, "temperature_ansys", grid_res=grid_res)
        X_pinn, Y_pinn, Z_pinn = interpolate_data(df_pinn, t, "temperature_pinn", grid_res=grid_res)

        # Debug: Check interpolated data range
        print(f"\nTime {t}:")
        print(f"ANSYS interpolated min={np.nanmin(Z_ansys):.2f}, max={np.nanmax(Z_ansys):.2f}")
        print(f"PINN interpolated min={np.nanmin(Z_pinn):.2f}, max={np.nanmax(Z_pinn):.2f}")

        # Compute absolute error map
        error_map = np.abs(Z_ansys - Z_pinn)

        # Determine shared vmin/vmax for ANSYS and PINN for this time step
        ansys_min = df_ansys[df_ansys["Time"] == t]["temperature_ansys"].min()
        ansys_max = df_ansys[df_ansys["Time"] == t]["temperature_ansys"].max()
        pinn_min = df_pinn[df_pinn["Time"] == t]["temperature_pinn"].min()
        pinn_max = df_pinn[df_pinn["Time"] == t]["temperature_pinn"].max()
        shared_vmin = min(ansys_min, pinn_min)
        shared_vmax = max(ansys_max, pinn_max)
        print(f"Time {t}: Shared vmin={shared_vmin:.2f}, vmax={shared_vmax:.2f}")

        # Create figure with three subplots
        fig, axs = plt.subplots(1, 3, figsize=(18, 5))

        # Define custom tick positions and labels for -10 to 10 mm
        tick_positions = np.linspace(0, grid_res-1, 5)  # 0, 50, 100, 150, 199 (grid indices)
        tick_labels = np.linspace(-10, 10, 5)  # -10, -5, 0, 5, 10 (mm)

        # ANSYS heatmap
        sns.heatmap(
            Z_ansys,
            ax=axs[0],
            cmap='jet',
            vmin=shared_vmin,
            vmax=shared_vmax,
            cbar=True,
            square=True
        )
        axs[0].set_title(f'ANSYS (Time = {t}s)')
        axs[0].set_xlabel("X (mm)")
        axs[0].set_ylabel("Y (mm)")
        axs[0].set_xticks(tick_positions)
        axs[0].set_xticklabels([f"{label:.0f}" for label in tick_labels])
        axs[0].set_yticks(tick_positions)
        axs[0].set_yticklabels([f"{label:.0f}" for label in tick_labels])
        axs[0].invert_yaxis()  # Ensures -10 is at the bottom, 10 at the top

        # PINN heatmap
        sns.heatmap(
            Z_pinn,
            ax=axs[1],
            cmap='jet',
            vmin=shared_vmin,
            vmax=shared_vmax,
            cbar=True,
            square=True
        )
        axs[1].set_title(f'PINN (Time = {t}s)')
        axs[1].set_xlabel("X (mm)")
        axs[1].set_ylabel("Y (mm)")
        axs[1].set_xticks(tick_positions)
        axs[1].set_xticklabels([f"{label:.0f}" for label in tick_labels])
        axs[1].set_yticks(tick_positions)
        axs[1].set_yticklabels([f"{label:.0f}" for label in tick_labels])
        axs[1].invert_yaxis()

        # Error heatmap
        error_max = np.nanmax(error_map)
        sns.heatmap(
            error_map,
            ax=axs[2],
            cmap='magma',
            vmin=0,
            vmax=error_max if error_max > 0 else 1,
            cbar=True,
            square=True
        )
        axs[2].set_title(f'Absolute Error (Time = {t}s)')
        axs[2].set_xlabel("X (mm)")
        axs[2].set_ylabel("Y (mm)")
        axs[2].set_xticks(tick_positions)
        axs[2].set_xticklabels([f"{label:.0f}" for label in tick_labels])
        axs[2].set_yticks(tick_positions)
        axs[2].set_yticklabels([f"{label:.0f}" for label in tick_labels])
        axs[2].invert_yaxis()

        # Adjust layout and save
        plt.tight_layout()
        plt.savefig(f'comparison_map_time_{t}.png', dpi=300, bbox_inches='tight')
        plt.close()

    except ValueError as e:
        print(f"Error processing time step {t}: {e}")
        continue

ANSYS raw data (first 5 rows):
   Time   x-coordinate  y-coordinate  temperature 
0      1        -0.008        -0.008         339.0
1      1        -0.008        -0.006         341.0
2      1        -0.008        -0.004         345.0
3      1        -0.008        -0.002         350.0
4      1        -0.008         0.000         356.0
ANSYS raw columns: ['Time ', 'x-coordinate', 'y-coordinate', 'temperature ']

ANSYS data before dropna:
   Time    x    y  temperature_ansys
0     1 -8.0 -8.0              339.0
1     1 -8.0 -6.0              341.0
2     1 -8.0 -4.0              345.0
3     1 -8.0 -2.0              350.0
4     1 -8.0  0.0              356.0
ANSYS Time values unique: [1 2 3 4 5 6 7 8]

ANSYS data after dropna:
   Time    x    y  temperature_ansys
0     1 -8.0 -8.0              339.0
1     1 -8.0 -6.0              341.0
2     1 -8.0 -4.0              345.0
3     1 -8.0 -2.0              350.0
4     1 -8.0  0.0              356.0
ANSYS data shape: (648, 4)

ANSYS data after 